In [1]:
import cvxpy as cp
import numpy as np

# ==============================================================================
# Chemical Equilibrium Problem (White, Johnson, Dantzig 1958)
# Mixture: 1/2 N2H4 + 1/2 O2 at 3500K, 750 psi
# ==============================================================================

n_compounds = 10
n_elements = 3

compounds = ["H", "H2", "H2O", "N", "N2", "NH", "NO", "O", "O2", "OH"]

# c_j = (F0/RT)_j + ln(P), where P = 750 psi
c = np.array([
    -6.089,   # H
    -17.164,  # H2
    -34.054,  # H2O
    -5.914,   # N
    -24.721,  # N2
    -14.986,  # NH
    -24.100,  # NO
    -10.708,  # O
    -26.662,  # O2
    -22.179   # OH
])

# Stoichiometry matrix A[i,j] = atoms of element i in compound j
# Rows: H, N, O
A = np.array([
    [1, 2, 2, 0, 0, 1, 0, 0, 0, 1],  # H
    [0, 0, 0, 1, 2, 1, 1, 0, 0, 0],  # N
    [0, 0, 1, 0, 0, 0, 1, 1, 2, 1],  # O (H2O has 1 oxygen atom)
], dtype=float)

# Atomic weights in mixture
b = np.array([2.0, 1.0, 1.0])

# ==============================================================================
# CVXPY Formulation
# ==============================================================================

# Decision variables: moles of each compound
x = cp.Variable(n_compounds, nonneg=True)

# Total moles
x_total = cp.sum(x)

# Objective: minimize Gibbs free energy
# f(x) = sum_j x_j * (c_j + ln(x_j / x_total))
#      = c @ x + sum_j x_j * ln(x_j) - x_total * ln(x_total)
#      = c @ x - sum(entr(x)) + entr(x_total)
objective = c @ x - cp.sum(cp.entr(x)) + cp.entr(x_total)

# Mass balance constraints
constraints = [A @ x == b]

# Problem
prob = cp.Problem(cp.Minimize(objective), constraints)
prob.solve(solver=cp.IPOPT, nlp=True, verbose=True)

# Output results
print("Optimal value:", prob.value)
for i in range(n_compounds):
    print(f"{compounds[i]}: {x.value[i]:.4f} moles")
    

(CVXPY) Dec 14 11:37:31 AM: Your problem has 10 variables, 3 constraints, and 0 parameters.
(CVXPY) Dec 14 11:37:31 AM: It is compliant with the following grammars: 
(CVXPY) Dec 14 11:37:31 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 14 11:37:31 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Dec 14 11:37:31 AM: Your problem is compiled with the CPP canonicalization backend.


                                     CVXPY                                     
                             v1.8.0.dev0+0.46d7c05                             

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.17, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:       45
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       11

Total number of variables............................:       21
                     variables with only lower bounds:       21
                variables with lower and upper bounds:      

In [2]:
import plotly.graph_objects as go
import plotly.express as px

# Extract solution
moles = x.value
total_moles = np.sum(moles)
mole_fractions = moles / total_moles

# Generate colors for compounds
colors = px.colors.qualitative.Set3[:n_compounds]

# Create bar chart with logarithmic scale
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=compounds,
        y=mole_fractions,
        marker=dict(
            color=colors
        ),
        hovertemplate='<b>%{x}</b><br>Mole Fraction: %{y:.4e}<extra></extra>'
    )
)

# Update layout with log scale and improved styling
fig.update_layout(
    xaxis={
        'title': 'Chemical Species',
        'title_font': {'size': 18},
        'tickfont': {'size': 14}
    },
    yaxis={
        'title': 'Mole Fraction',
        'title_font': {'size': 18},
        'type': 'log',
        'tickfont': {'size': 14},
        'gridcolor': 'rgba(128,128,128,0.2)'
    },
    height=600,
    width=1000,
    plot_bgcolor='white',
    paper_bgcolor='white',
    hovermode='x unified'
)

# Save to PDF file
output_file = 'chemical_equilibrium_plot.pdf'
fig.write_image(output_file)
print(f"Plot saved to {output_file}")

# Print summary
print(f"\nTotal moles at equilibrium: {total_moles:.4f}")
print(f"Gibbs free energy (minimized): {prob.value:.4f}")
print(f"\nMole fractions:")
for compound, frac in zip(compounds, mole_fractions):
    print(f"  {compound:5s}: {frac:.4e}")

# Display figure
fig

Plot saved to chemical_equilibrium_plot.pdf

Total moles at equilibrium: 1.6384
Gibbs free energy (minimized): -47.7611

Mole fractions:
  H    : 2.4821e-02
  H2   : 9.0165e-02
  H2O  : 4.7799e-01
  N    : 8.6315e-04
  N2   : 2.9616e-01
  NH   : 4.2307e-04
  NO   : 1.6723e-02
  O    : 1.0954e-02
  O2   : 2.2774e-02
  OH   : 5.9124e-02
